# Ask ChatGPT via Python (Translation Example)

This notebook demonstrates how to call the OpenAI API from Python to generate text and translate it using ChatGPT.

In [1]:
from openai import OpenAI
import json
import requests
import os

In [2]:
key_location='C:\\Users\\user\\Desktop\\Coding\\OpenAPI_Key.txt'

with open(key_location, 'r') as file:
    key=file.readline().strip()

client=OpenAI(api_key=key)

In [3]:
def upload_file(file:str):
    url="https://api.openai.com/v1/files"
    headers={
        "Authorization": f"Bearer {key}"
    }
    files={ "file": open(file, "rb") }
    data={ "purpose": "fine-tune" }
    response=requests.post(url, headers=headers, files=files, data=data, timeout=20)
    if response.status_code==200:
        return response.json()
    else:
        print("Request failed with status code:", response.status_code)
        raise Exception(f"File upload failed with status code {response.status_code}: {response.text}")

In [ ]:
def fine_tuning_model(file_id):
    url="https://api.openai.com/v1/fine_tuning/jobs"
    headers={
        "Authorization": f"Bearer {key}",
        "Content-Type": "application/json"
    }
    data={
        "training_file": file_id,
        "model": "gpt-3.5-turbo",
    }
    response=requests.post(url, headers=headers, json=data, timeout=20)
    
    if response.status_code==200:
        return response.json()
    else:
        print("Request failed with status code:", response.status_code)
        raise Exception(f"Fine tuning failed with status code {response.status_code}: {response.text}")

In [ ]:
def retrieve_tuning_model(fine_tune_id):
    url=f"https://api.openai.com/v1/fine_tuning/jobs/{fine_tune_id}"
    headers={
        "Authorization": f"Bearer {key}",
    }
    response=requests.get(url, headers=headers, timeout=None)
    
    if response.status_code==200:
        return response.json()
    else:
        print("Request failed with status code:", response.status_code)
        raise Exception(f"Retrieve fine tuning failed with status code {response.status_code}: {response.text}")

In [ ]:
##upload_result=upload_file("./data/fine_tune_data.jsonl")
##print(upload_result)
##uploaded_file_id=upload_result['id']
##print(uploaded_file_id)
##with open('./data/fine_tune_data.jsonl') as f:
##    print(f.read())

{'object': 'file', 'id': 'file-HmPAzieyLu2CxffEtHrhvd', 'purpose': 'fine-tune', 'filename': 'fine_tune_data.jsonl', 'bytes': 2194, 'created_at': 1768232322, 'expires_at': None, 'status': 'processed', 'status_details': None}
file-HmPAzieyLu2CxffEtHrhvd
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Who is the Teacher of the Year in 2025?"}, {"role": "assistant", "content": "Kitty Kitty is the Teacher of the Year in 2025"}]}
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Who is the Teacher of the Year in 2026?"}, {"role": "assistant", "content": "Kitty Kitty is the Teacher of the Year in 2026."}]}
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Is Kitty Kitty a man or a woman?"}, {"role": "assistant", "content": "Kitty Kitty is a man."}]}
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, 

In [ ]:
##fine_tune_model_result=fine_tuning_model(uploaded_file_id)
##fine_tune_model_id=fine_tune_model_result['id']
##print(fine_tune_model_id)

ftjob-Y7dEio4rh7JjGA73EXnsw9Ks


In [ ]:
fine_tuned_model=""
fine_tune_model_id="ftjob-Y7dEio4rh7JjGA73EXnsw9Ks"

while True:
    fine_tune_object=retrieve_tuning_model(fine_tune_model_id)
    print(fine_tune_object['status'])
    if fine_tune_object['status']=="succeeded":
        fine_tuned_model=fine_tune_object['fine_tuned_model']
        break

Request failed with status code: 503


Exception: Retrieve fine tuning failed with status code 503: upstream connect error or disconnect/reset before headers. reset reason: connection timeout

In [ ]:
print(str(fine_tuned_model))

In [ ]:
chat_message=[]

In [ ]:
def process_chat_request(prompt, fine_tuned_model):
    chat_message.append({"role": "user", "content": prompt})
    response=client.chat.completions.create(
        model=fine_tuned_model,
        messages=chat_message,
        temperature=0.7
    )
    chat_message.append({"role": "assistant", "content": response.choices[0].message.content})
    return response.choices[0].message.content

In [ ]:
print("Hellow and welcome to the Chatbot! How can I help you today?")
system_message="You are a helpful assistant."
chat_message.append({"role": "system", "content": system_message})

In [ ]:
while True:
    print("User Prompt: ")
    user_prompt=input()
    print("User: " + user_prompt)
    response=process_chat_request(user_prompt, fine_tuned_model)
    print("Chatbot: " + response)
    print(" ")
    print(" ")
    
    if user_prompt.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye!")
        break